# Terms of Trade - Data Prep
This notebook contains data preparation for IMF Terms of Trade data. Objective is to generate a cleaned parquet file to store in our S3 analytics bucket.

In [1]:
import json
import pandas as pd

In [2]:
path = '/path/to/data'

In [3]:
# Load raw
df = pd.read_csv(f'{path}/terms_of_trade.csv')

In [4]:
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Type Name,Type Code,Attribute,1962,1963,1964,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 64
0,Greece,174,"Commodity Import Price Index, Individual Commo...",m,"Historical, Fixed Weights, Index",H_FW_IX,Value,32.692356,32.278507,32.763863,...,87.834663,105.536720,100,102.137352,98.743393,65.697014,61.649487,72.005081,82.075478,NaN
1,Greece,174,"Commodity Net Export Price Index, Individual C...",xm,"Historical, Fixed Weights, Index",H_FW_IX,Value,176.172150,176.750992,175.789520,...,107.534180,99.887138,100,99.248283,100.296738,119.767647,124.192398,116.796890,110.653725,NaN
2,Greece,174,"Commodity Export Price Index, Individual Commo...",x_gdp,"Historical, Fixed Weights, Index",H_FW_IX,Value,97.682693,97.637894,97.670189,...,99.790306,100.283089,100,100.062904,99.937904,98.909447,98.809395,99.242813,99.605728,NaN
3,Greece,174,"Commodity Import Price Index, Individual Commo...",m_gdp,"Historical, Fixed Weights, Index",H_FW_IX,Value,93.085983,93.021774,93.076469,...,99.182724,100.295761,100,100.120567,99.893982,97.399086,96.993805,97.921791,98.744637,NaN
4,Greece,174,"Commodity Net Export Price Index, Individual C...",xm_gdp,"Historical, Fixed Weights, Index",H_FW_IX,Value,104.938110,104.962387,104.935394,...,100.612587,99.987366,100,99.942406,100.043961,101.550682,101.871864,101.349060,100.872040,NaN


In [5]:
# Reshape data
id_vars = df.columns[:7]
values = df.columns[7:-1]
df = df.melt(id_vars=id_vars, value_vars=values, var_name='year')

In [6]:
# Rename columns
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [7]:
df.head()

,country_name,country_code,indicator_name,indicator_code,type_name,type_code,attribute,year,value
0,Greece,174,"Commodity Import Price Index, Individual Commo...",m,"Historical, Fixed Weights, Index",H_FW_IX,Value,1962,32.692356
1,Greece,174,"Commodity Net Export Price Index, Individual C...",xm,"Historical, Fixed Weights, Index",H_FW_IX,Value,1962,176.172150
2,Greece,174,"Commodity Export Price Index, Individual Commo...",x_gdp,"Historical, Fixed Weights, Index",H_FW_IX,Value,1962,97.682693
3,Greece,174,"Commodity Import Price Index, Individual Commo...",m_gdp,"Historical, Fixed Weights, Index",H_FW_IX,Value,1962,93.085983
4,Greece,174,"Commodity Net Export Price Index, Individual C...",xm_gdp,"Historical, Fixed Weights, Index",H_FW_IX,Value,1962,104.938110


In [8]:
# Join ISO country mapping
with open(f'{path}/iso_codes_3166.json' ,'r') as f:
    countries = pd.DataFrame(json.load(f))
df = df.merge(countries)

In [9]:
# Define relevant column, subset and save to file
cols = ['code', 'country_name', 'indicator_name', 'indicator_code', 'type_name', 'type_code', 'year', 'value']
df = df[cols]
df.to_parquet(f'{path}/terms_of_trade.parquet')

In [10]:
df.head()

,code,country_name,indicator_name,indicator_code,type_name,type_code,year,value
0,GRC,Greece,"Commodity Import Price Index, Individual Commo...",m,"Historical, Fixed Weights, Index",H_FW_IX,1962,32.692356
1,GRC,Greece,"Commodity Net Export Price Index, Individual C...",xm,"Historical, Fixed Weights, Index",H_FW_IX,1962,176.172150
2,GRC,Greece,"Commodity Export Price Index, Individual Commo...",x_gdp,"Historical, Fixed Weights, Index",H_FW_IX,1962,97.682693
3,GRC,Greece,"Commodity Import Price Index, Individual Commo...",m_gdp,"Historical, Fixed Weights, Index",H_FW_IX,1962,93.085983
4,GRC,Greece,"Commodity Net Export Price Index, Individual C...",xm_gdp,"Historical, Fixed Weights, Index",H_FW_IX,1962,104.938110
